In [50]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
from statistics import mode
%config InlineBackend.figure_format = 'retina'

In [123]:
df = pd.read_csv('sentiment_data/chrisbrown.csv')
df = df.drop(df[df.release_date == 'None'].index, axis=0)
df = df.reset_index().drop(['index', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df

,api_path,primary_artist,title,url,song_endpoint,album,release_date,lyrics,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,polarity
0,/songs/62387,Chris Brown,100%,https://genius.com/Chris-brown-100-lyrics,http://api.genius.com/songs/62387,Boy In Detention (Mixtape),2011-08-05,"Beats I be servin', Saks Fifth splurgin' Diam...",14,13,5,7,12,11,24,5,11,17,-0.9809
1,/songs/54007,Chris Brown,100 Bottles,https://genius.com/Chris-brown-100-bottles-lyrics,http://api.genius.com/songs/54007,Boy In Detention (Mixtape),2011-08-05,We in the motherfuckin' buildin' 100 fuckin...,19,1,16,15,1,20,13,16,1,4,-0.9967
2,/songs/516349,Chris Brown,101 (Interlude),https://genius.com/Chris-brown-101-interlude-l...,http://api.genius.com/songs/516349,X,2014-09-16,You got me doing 101 on the 101 Don't go nowh...,2,4,2,1,6,6,11,2,2,6,0.9516
3,/songs/3103904,Chris Brown,10 Feet,https://genius.com/Chris-brown-10-feet-lyrics,http://api.genius.com/songs/3103904,None,2017-05-31,Ain't no way we could ignore this fire When I...,0,0,0,4,0,3,4,2,0,2,0.8592
4,/songs/57217,Chris Brown,2011 BET Cypher Chris Brown and Friends,https://genius.com/Chris-brown-2011-bet-cypher...,http://api.genius.com/songs/57217,None,2019-04-04,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",20,17,11,12,14,23,39,9,17,25,0.9745
5,/songs/79425,Chris Brown,2012,https://genius.com/Chris-brown-2012-lyrics,http://api.genius.com/songs/79425,Fortune,2012-06-29,Faire l'amour lendemain What if the sky was...,3,11,1,12,11,8,14,2,2,4,0.8478
6,/songs/60609,Chris Brown,2 Complicated,https://genius.com/Chris-brown-2-complicated-l...,http://api.genius.com/songs/60609,None,2012-06-19,Shattered glasses and cigarettes There's a lo...,8,10,8,11,14,37,13,9,13,13,-0.9576
7,/songs/3089879,Chris Brown,2 the Side,https://genius.com/Chris-brown-2-the-side-lyrics,http://api.genius.com/songs/3089879,None,2017-05-28,"Oh, oh, oh, babe, yeah I know what you need, ...",6,3,5,2,15,13,47,3,3,23,-0.1984
8,/songs/2404874,Chris Brown,45,https://genius.com/Chris-brown-45-lyrics,http://api.genius.com/songs/2404874,None,2017-05-31,And I'm like Jordan wearin' 45 I told you I w...,16,7,16,15,4,27,18,20,7,26,-0.9491
9,/songs/2877,Chris Brown,48 Bar Rap,https://genius.com/Chris-brown-48-bar-rap-lyrics,http://api.genius.com/songs/2877,Fan of a Fan,2010-05-16,"Yeah, leggo It's what they wanted right? Bl...",13,10,9,9,7,15,15,9,7,9,-0.7584


In [124]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year'] = df['release_date'].dt.year

In [125]:
df = df.drop(df[df.album == 'None'].index, axis=0)

In [126]:
albums = df[['release_year', 'album']]
albums = albums.groupby('album')[['release_year']].max().reset_index().sort_values(by='release_year')

In [127]:
songs = df[['album', 'title']]
albums_list = albums.album.tolist()
albums_of_songs = {}

for album in albums_list:
    songs_list = []
    for i, row in songs.iterrows():
        if album == row.album:
            songs_list.append(row.title)
    if len(songs_list) > 20:
        songs_list = random.sample(songs_list, 10)
    albums_of_songs[album] = songs_list  

In [128]:
albums['songs'] = list(albums_of_songs.values())

In [129]:
albums

,album,release_year,songs
21,Usher Presents In the Mix: Music from the Moti...,2006,[Which One]
5,Chris Brown,2006,"[Ain't No Way (You Won't Love Me), Gimme That,..."
20,This Christmas - Songs From The Motion Picture,2007,"[This Christmas, Try a Little Tenderness]"
0,AT&T Team USA Soundtrack,2008,[Dreamer]
6,Exclusive,2008,"[Damage, Down, Fallen Angel, Get At Ya, Gimme ..."
7,Exclusive (The Forever Edition),2008,"[Forever, Heart Ain't a Brain, Superhuman]"
9,Fan of a Fan,2010,[48 Bar Rap]
11,Graffiti,2010,"[I'll Go, Fallin' Down, Famous Girl, So Cold, ..."
15,In My Zone 2 (Mixtape),2010,"[All Off, Another You, Awol, Boing, Christmas ..."
4,Boy In Detention (Mixtape),2011,"[100%, 100 Bottles, Body on Mine, Crazy, First..."


In [130]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Panel, Tabs, ColumnDataSource, LabelSet

In [139]:
output_notebook()

a = int(albums['release_year'].min())
b = int(albums['release_year'].max())

source = ColumnDataSource(albums)
    
p = figure(plot_width=1000, plot_height=200, x_range=(a-1, b+1), y_range=(0, 0.5), x_axis_label='Years'
           , toolbar_location='above')

p.scatter(albums['release_year'], 0.25, marker='square_cross', fill_color='#D3D3D3', line_color='#696969', line_width=1.5, size=15)

h = p.scatter('release_year', 0.25, marker='square', source=albums, size=35, fill_color='white', hover_fill_color='#DCDCDC', fill_alpha=0,
                   hover_alpha=0.2, line_color=None, hover_line_color='#DCDCDC')

tools = [("album", "@album"), ("songs", "@songs")]

p.add_tools(HoverTool(tooltips=tools, renderers=[h], mode='mouse'))

p.xaxis.major_tick_line_color = 'firebrick'
p.xaxis.major_tick_line_width = 5
p.xaxis.minor_tick_line_color = 'orange'
p.axis.major_tick_out = 10
p.axis.minor_tick_out = 8
p.yaxis.major_tick_line_color = None  
p.yaxis.minor_tick_line_color = None 
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None
p.outline_line_color = None
p.xgrid[0].ticker.desired_num_ticks = 10 
p.yaxis.visible = False
show(p)

Loading BokehJS ...

In [141]:
a = int(df['release_year'].min())
b = int(df['release_year'].max())
source = ColumnDataSource(df)
# p = figure(plot_width=1000, plot_height=250, x_range=(a-1, b+1), y_range=(0, 0.5), toolbar_location='above', tools = "reset")
# p.scatter('release_year', 0.25, source=df, marker='square_cross', fill_color='#D3D3D3', line_color='#696969', line_width=1.5, size=15)
# h = p.scatter('release_year', 0.25, marker='square', source=df, size=35, fill_color='white', hover_fill_color='#DCDCDC', fill_alpha=0,
#                hover_alpha=0.2, line_color=None, hover_line_color='#DCDCDC')

p = figure(plot_width=1000, plot_height=200, x_range=(a-1, b+1), y_range=(0, 0.5), x_axis_label='Years'
           , toolbar_location='above')

p.scatter(albums['release_year'], 0.25, marker='square_cross', fill_color='#D3D3D3', line_color='#696969', line_width=1.5, size=15)

h = p.scatter('release_year', 0.25, marker='square', source=albums, size=35, fill_color='white', hover_fill_color='#DCDCDC', fill_alpha=0,
                   hover_alpha=0.2, line_color=None, hover_line_color='#DCDCDC')



TOOLTIPS = """
<div>
    <div>
        <span style="font-size: 15px;">Album: </span>
        <span style="font-size: 17px; font-weight: bold;">@album</span>
    </div>
    <div>
        <span style="font-size: 15px;">Songs: </span>
        <span style="font-size: 17px; color: gray;">@songs</span>
    </div>
</div>
"""

p.add_tools(HoverTool(tooltips=TOOLTIPS, renderers=[h], mode='mouse'))

p.xaxis.major_tick_line_color = 'firebrick'
p.xaxis.major_tick_line_width = 5
p.xaxis.minor_tick_line_color = 'orange'
p.axis.major_tick_out = 10
p.axis.minor_tick_out = 8
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None
p.outline_line_color = None
p.xgrid[0].ticker.desired_num_ticks = 8
p.yaxis.visible = False
show(p)